В решении задачи в качестве исходного дата-сета используйте CSV-файл https://video.ittensive.com/analytics-way/2022-04-22.dataset.csv.

На входе у вас есть таблица orders, содержащая все заказы, совершенные в ОЗОН<br>
Описание таблицы orders:<br>
* user_id (string) - уникальный идентификатор пользователя
* order_id (string) - уникальный идентификатор заказа
* order_time (Int64) - время сделанного заказа в unixtime (секунды, UTC)
* order_cost (float) - стоимость заказа
* success_order_flg (bool) - идентификатор, определяющий был ли заказ успешно доставлен. Выполненный заказ - это заказ у которого в данной таблице success_order_flg=True

Пользователь считается привлеченным в день date, если он совершает первый заказ в день date.

Пользователь считается реактивированным в день date, если он совершает заказ в день date, при этом не совершил ни одного заказа в течении предшествующих 90 дней, но совершал хотя бы 1 заказ за свою пользовательскую историю ранее.

На выходе требуется получить таблицу:
* date - календарная дата привлечения или реактивации
* gmv360d_new - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, привлеченными в день date
* gmv360d_reactivated - общая стоимость заказов за 360 дней после дня date, совершенных пользователями, реактивированными в день date
* users_count_new - кол-во пользователей, привлеченных в день date
* users_count_reactivated- кол-во пользователей, реактивированных в день date

Заказы, совершенные пользователем после очередной реактивации, не должны учитываться в расчете GMV для предыдущих периодов активности пользователя. Т.е., например, если пользователь пришел в день 0, совершал заказы по день 30, не совершал заказов до дня 130 и совершил заказ в день 130, его заказы в дни 130-490 должны учитываться только в gmv360d_reactivated для дня 130, но не в gmv360d_new дня 0.

То же правило распространяется на повторные реактивации пользователя. То есть если у рассмотренного нами пользователя на 230-й день жизни снова произошла реактивация, то gmv360d_reactivated будет посчитано для предыдущего периода за дни 130-229, а для итогового, с реактивацией на 230-й день, это поле будет рассчитано за дни 230-590.

Сколько пользователей было реактивировано 20 марта 2020 года?

In [8]:
# загрузка необходимых библиотек

import pandas as pd
from datetime import datetime

Загрузка данных, добавление даты заказа в order_data

In [51]:
data = pd.read_csv('https://video.ittensive.com/analytics-way/2022-04-22.dataset.csv', sep=';')
print(data.head())

   user_id   order_id  order_time  order_cost  success_order_flg
0  1728898  118881751  1578345249  12804.1667                  0
1  1728898  132150404  1584367397   5147.2728                  1
2  1728898  133654929  1585501200   8600.0000                  1
3  1728898  133770763  1585235169   6807.3031                  1
4  1766832  118492969  1578582566   2625.0000                  1


In [52]:
data['order_date'] = pd.to_datetime(data['order_time'] ,unit='s').dt.date
# data['order_date'].astype('datetime64[D]')

In [53]:
data.head(10)

,user_id,order_id,order_time,order_cost,success_order_flg,order_date
0,1728898,118881751,1578345249,12804.1667,0,2020-01-06
1,1728898,132150404,1584367397,5147.2728,1,2020-03-16
2,1728898,133654929,1585501200,8600.0000,1,2020-03-29
3,1728898,133770763,1585235169,6807.3031,1,2020-03-26
4,1766832,118492969,1578582566,2625.0000,1,2020-01-09
5,1766832,120888914,1579343582,2408.0000,0,2020-01-18
6,1766832,124602594,1581249228,7494.0000,1,2020-02-09
7,1766832,124604579,1581498821,2913.3332,1,2020-02-12
8,1766832,125072398,1581278385,1765.5001,1,2020-02-09
9,1766832,125198531,1581331692,1536.6668,1,2020-02-10


In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56680 entries, 0 to 56679
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            56680 non-null  int64  
 1   order_id           56680 non-null  int64  
 2   order_time         56680 non-null  int64  
 3   order_cost         56680 non-null  float64
 4   success_order_flg  56680 non-null  int64  
 5   order_date         56680 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 2.6+ MB
